##### Model Deployment

#### Generate new fake data (4500 rows)

In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
import time

# Original dataset
df = pd.read_csv('Extended_Employee_Performance_and_Productivity_Data.csv')
df['Hire_Date'] = pd.to_datetime(df['Hire_Date'], dayfirst=True).dt.strftime('%d/%m/%Y')

# Initialize Faker for realistic fake data
fake = Faker()

# Precompute department-job title mappings for faster access
dept_job_mapping = df.groupby('Department')['Job_Title'].unique().to_dict()
job_edu_mapping = df.groupby('Job_Title')['Education_Level'].unique().to_dict()
job_perf_salary_mapping = df.groupby(['Job_Title', 'Performance_Score'])['Monthly_Salary'].apply(list).to_dict()

def generate_new_records_fast(n):
    emp_ids = np.arange(1000000, 1000000 + n)
    
    # Sample departments and genders
    departments = np.random.choice(df['Department'].unique(), size=n)
    genders = np.random.choice(df['Gender'].unique(), size=n)
    
    # Get job titles based on departments
    job_titles = np.array([np.random.choice(dept_job_mapping[dept]) for dept in departments])
    
    # Generate ages, years at company within 1 years
    ages = np.random.randint(20, 60, size=n)
    years_at_company = np.random.randint(0, 1, size=n)
    
    # Generate hire dates between July 1, 2024 to July 1, 2025 
    start_date = pd.to_datetime('2024-07-01')
    end_date = pd.to_datetime('2025-07-01')
    date_range = (end_date - start_date).days
    random_days = np.random.randint(0, date_range + 1, size=n)  
    hire_dates = pd.Series(start_date + pd.to_timedelta(random_days, unit='D')).dt.strftime('%d/%m/%Y')
    
    # Get education levels based on job titles
    educations = np.array([np.random.choice(job_edu_mapping.get(job, df['Education_Level'].unique())) 
                         for job in job_titles])
    
    # Generate performance scores
    performances = np.random.choice([1, 2, 3, 4, 5], size=n)
    
    # Get salaries based on job title and performance
    salaries = np.array([
        np.random.choice(job_perf_salary_mapping.get((job, perf), df[df['Job_Title'] == job]['Monthly_Salary'].values))
        for job, perf in zip(job_titles, performances)
    ])
    
    # Generate other fields
    work_hours = np.random.randint(30, 60, size=n)
    projects = np.random.randint(0, 50, size=n)
    overtime = np.random.randint(0, 30, size=n)
    sick_days = np.random.randint(0, 15, size=n)
    remote_freq = np.random.randint(30, 80, size=n)
    team_size = np.random.randint(1, 20, size=n)
    training_hours = np.random.randint(0, 20, size=n)
    promotions = np.random.randint(0, 3, size=n)
    satisfaction = np.round(np.random.uniform(1, 5, size=n), 2)
    
    new_data = pd.DataFrame({
        'Employee_ID': emp_ids,
        'Department': departments,
        'Gender': genders,
        'Age': ages,
        'Job_Title': job_titles,
        'Hire_Date': hire_dates,
        'Years_At_Company': years_at_company,
        'Education_Level': educations,
        'Performance_Score': performances,
        'Monthly_Salary': salaries,
        'Work_Hours_Per_Week': work_hours,
        'Projects_Handled': projects,
        'Overtime_Hours': overtime,
        'Sick_Days': sick_days,
        'Remote_Work_Frequency': remote_freq,
        'Team_Size': team_size,
        'Training_Hours': training_hours,
        'Promotions': promotions,
        'Employee_Satisfaction_Score': satisfaction,
        'Resigned': False
    })
    
    return new_data

start_time = time.time()

# Generate 4,500 new records with July 2024-July 2025 hire dates
new_records = generate_new_records_fast(4500)

# Combine with original data
extended_df = pd.concat([df, new_records], ignore_index=True)
extended_df['Hire_Date'] = pd.to_datetime(extended_df['Hire_Date'], dayfirst=True)

# Create Resignation_Date column (initially empty)
extended_df['Resignation_Date'] = pd.NaT

# Get all resigned employees
resigned_mask = extended_df['Resigned'] == True
resigned = extended_df[resigned_mask]

# Randomly select 805 for Q1 (Jan-Mar 2025) and 478 for Q2 (Apr-Jun 2025)
q1_resignations = resigned.sample(805, random_state=42)
remaining_resigned = resigned.drop(q1_resignations.index)
q2_resignations = remaining_resigned.sample(478, random_state=42)
other_resignations = remaining_resigned.drop(q2_resignations.index)

# Assign Q1 resignation dates (Jan-Mar 2025)
q1_start = pd.to_datetime('01/01/2025', dayfirst=True)
q1_end = pd.to_datetime('31/03/2025', dayfirst=True)
days_in_q1 = (q1_end - q1_start).days
random_days = np.random.randint(0, days_in_q1 + 1, size=len(q1_resignations))
extended_df.loc[q1_resignations.index, 'Resignation_Date'] = q1_start + pd.to_timedelta(random_days, unit='D')

# Assign Q2 resignation dates (Apr-Jun 2025)
q2_start = pd.to_datetime('01/04/2025', dayfirst=True)
q2_end = pd.to_datetime('30/06/2025', dayfirst=True)
days_in_q2 = (q2_end - q2_start).days
random_days = np.random.randint(0, days_in_q2 + 1, size=len(q2_resignations))
extended_df.loc[q2_resignations.index, 'Resignation_Date'] = q2_start + pd.to_timedelta(random_days, unit='D')

# Assign random past dates for other resignations (before 2025)
past_start = pd.to_datetime('01/01/2018', dayfirst=True)  
past_end = pd.to_datetime('31/12/2024', dayfirst=True)
days_in_past = (past_end - past_start).days
random_days = np.random.randint(0, days_in_past + 1, size=len(other_resignations))
extended_df.loc[other_resignations.index, 'Resignation_Date'] = past_start + pd.to_timedelta(random_days, unit='D')
extended_df['Hire_Date'] = extended_df['Hire_Date'].dt.strftime('%d/%m/%Y')
extended_df['Resignation_Date'] = extended_df['Resignation_Date'].dt.strftime('%d/%m/%Y')

extended_df.to_csv('Deployment_Used_Extended_Dataset_with_Resignation.csv', index=False)

print(f"Execution completed in {time.time() - start_time:.2f} seconds")
print("\nResignation date distribution:")
print(extended_df[resigned_mask]['Resignation_Date'].value_counts().sort_index())
print(f"\nTotal records: {len(extended_df)}")
print(f"Resigned employees: {resigned_mask.sum()}")
print(f"  - Q1 2025 resignations (Jan-Mar): {len(q1_resignations)}")
print(f"  - Q2 2025 resignations (Apr-Jun): {len(q2_resignations)}")
print(f"  - Other resignations: {len(other_resignations)}")
print(f"Current employees (no resignation date): {len(extended_df) - resigned_mask.sum()}")

Execution completed in 55.71 seconds

Resignation date distribution:
Resignation_Date
01/01/2018    14
01/01/2019    10
01/01/2020    19
01/01/2021    18
01/01/2022    19
              ..
31/12/2020    17
31/12/2021    20
31/12/2022    20
31/12/2023    16
31/12/2024    19
Name: count, Length: 2738, dtype: int64

Total records: 104500
Resigned employees: 42578
  - Q1 2025 resignations (Jan-Mar): 805
  - Q2 2025 resignations (Apr-Jun): 478
  - Other resignations: 41295
Current employees (no resignation date): 61922


#### Print columns

In [ ]:
import os
import pandas as pd 
from tabulate import tabulate

# Define the file path
file_path = "Deployment_Used_Extended_Dataset_with_Resignation.csv"

if os.path.isfile(file_path):
    df = pd.read_csv(file_path)
    print("Dataset loaded successfully!")
    print("\nFirst five rows of the dataset:")
    print(tabulate(df.head(), headers='keys', tablefmt='pretty', showindex=False))
else:
    print(f"Error: The file '{file_path}' was not found!")


Dataset loaded successfully!

First five rows of the dataset:
+-------------+------------------+--------+-----+------------+------------+------------------+-----------------+-------------------+----------------+---------------------+------------------+----------------+-----------+-----------------------+-----------+----------------+------------+-----------------------------+----------+------------------+
| Employee_ID |    Department    | Gender | Age | Job_Title  | Hire_Date  | Years_At_Company | Education_Level | Performance_Score | Monthly_Salary | Work_Hours_Per_Week | Projects_Handled | Overtime_Hours | Sick_Days | Remote_Work_Frequency | Team_Size | Training_Hours | Promotions | Employee_Satisfaction_Score | Resigned | Resignation_Date |
+-------------+------------------+--------+-----+------------+------------+------------------+-----------------+-------------------+----------------+---------------------+------------------+----------------+-----------+-----------------------+---

#### Show the currnt columns

In [4]:
print(df.columns)

Index(['Employee_ID', 'Department', 'Gender', 'Age', 'Job_Title', 'Hire_Date',
       'Years_At_Company', 'Education_Level', 'Performance_Score',
       'Monthly_Salary', 'Work_Hours_Per_Week', 'Projects_Handled',
       'Overtime_Hours', 'Sick_Days', 'Remote_Work_Frequency', 'Team_Size',
       'Training_Hours', 'Promotions', 'Employee_Satisfaction_Score',
       'Resigned', 'Resignation_Date'],
      dtype='object')
